In [1]:
# import libraries

import glob
import pandas as pd
import pandasql as ps
import numpy as np

### This notebook analyzes Covid death toll data using SQL queries with pandasql 

##### Data taken from:  https://github.com/owid/covid-19-data/tree/master/public/data

In [3]:
# owid-covid-19_data:downloaded on 2022-07-24

# Read data from excel sheet
covid_data = pd.read_excel("./data/owid-covid-data.xlsx")

In [4]:
# We split the data into two tables/dfs 
#   1. for covid deaths for all country
#   2. for Vaccine rate in all country

#first_table for death in each country
# It doesnot have data (all NULL) from North korea so we remove it 

cov_death  = covid_data.iloc[:,np.r_[0:24,48]]

cov_death.tail() 
cov_death.shape

(203567, 25)

In [5]:
#Second_table for vaccine rate

cov_vaccine=covid_data.iloc[:,24:] 

cov_vaccine.tail(3)
cov_vaccine.shape

(203567, 43)

In [6]:
#SQL query : total cases vs deaths; finds out death_rate 
#Location column has also continent as country-name so remove it by conditional phrase continent is not null 


death_percent_query =  """
   SELECT location, date, total_cases, total_deaths, (total_deaths/total_cases)*100 
   as death_percent 
   FROM cov_death WHERE continent is not null ORDER BY date DESC 
   """

ps.sqldf(death_percent_query)   

,location,date,total_cases,total_deaths,death_percent
0,Afghanistan,2022-07-24,184587.0,7738.0,4.192061
1,Albania,2022-07-24,293917.0,3517.0,1.196596
2,Algeria,2022-07-24,266839.0,6876.0,2.576835
3,Andorra,2022-07-24,45326.0,153.0,0.337555
4,Angola,2022-07-24,102301.0,1912.0,1.868994
...,...,...,...,...,...
191773,Mexico,2020-01-03,NaN,NaN,NaN
191774,Argentina,2020-01-02,NaN,NaN,NaN
191775,Mexico,2020-01-02,NaN,NaN,NaN
191776,Argentina,2020-01-01,NaN,NaN,NaN


In [7]:
#SQL query : find out the time when death rate was maximum across all countries 
# doesnot have data from North korea

death_percent_max_query = """
     SELECT location, date, total_cases , total_deaths, max((total_deaths)/(total_cases)*100) 
     as max_death_rate
     FROM cov_death WHERE continent is not null
     GROUP BY location ORDER BY max_death_rate DESC
 """

ps.sqldf(death_percent_max_query) 

,location,date,total_cases,total_deaths,max_death_rate
0,North Korea,2022-05-14,1.0,6.0,600.0
1,Sudan,2020-03-14,1.0,1.0,100.0
2,San Marino,2020-02-29,1.0,1.0,100.0
3,Peru,2020-03-06,1.0,1.0,100.0
4,Iran,2020-02-19,2.0,2.0,100.0
...,...,...,...,...,...
226,Marshall Islands,2022-07-24,59.0,NaN,NaN
227,Jersey,2022-07-17,NaN,NaN,NaN
228,Guernsey,2022-07-12,NaN,NaN,NaN
229,Guam,2022-06-18,NaN,NaN,NaN


In [8]:
#SQL query :to find out max_death rate in US :to find out the time when death rate in US was max

death_percent_max_US = """
  SELECT date,location,total_cases, total_deaths, max((total_deaths/total_cases)*100)
  as max_death_percent_until_now 
  FROM cov_death WHERE location like '%states'
  """

ps.sqldf(death_percent_max_US)  

,date,location,total_cases,total_deaths,max_death_percent_until_now
0,2020-03-02,United States,55.0,6.0,10.909091


In [9]:
##infection_rate_per_population on 2022-07-24(today)

infection_rate_query ="""
    SELECT location, date, new_cases, population, (new_cases/population)*100 
    as infection_rate_per_population 
    FROM cov_death WHERE date='2022-07-24' and 
    continent is not null
    ORDER BY 1, 2 DESC
    """

ps.sqldf(infection_rate_query) 

,location,date,new_cases,population,infection_rate_per_population
0,Afghanistan,2022-07-24,114.0,40099462.0,0.000284
1,Albania,2022-07-24,0.0,2854710.0,0.000000
2,Algeria,2022-07-24,67.0,44177969.0,0.000152
3,Andorra,2022-07-24,NaN,79034.0,NaN
4,Angola,2022-07-24,0.0,34503774.0,0.000000
...,...,...,...,...,...
211,Vietnam,2022-07-24,748.0,97468028.0,0.000767
212,Wallis and Futuna,2022-07-24,0.0,11627.0,0.000000
213,Yemen,2022-07-24,0.0,32981641.0,0.000000
214,Zambia,2022-07-24,NaN,19473125.0,NaN


In [10]:
#max_infected_rate_per_population:finds out the max_infection rate and the time of that infection rate across diff countries

max_infection_rate_query ="""
    SELECT location, date, new_cases, population, max((new_cases/population)*100) 
    as max_infection_rate_per_population 
    FROM cov_death WHERE continent is not null
    GROUP BY location ORDER BY max((new_cases/population)*100)  DESC
    """

ps.sqldf(max_infection_rate_query) 

,location,date,new_cases,population,max_infection_rate_per_population
0,Falkland Islands,2022-05-12,734.0,3764.0,19.500531
1,Bonaire Sint Eustatius and Saba,2021-04-29,1360.0,26706.0,5.092489
2,Saint Pierre and Miquelon,2022-04-01,279.0,5883.0,4.742478
3,Isle of Man,2022-01-04,3221.0,84263.0,3.822556
4,Montserrat,2022-05-10,160.0,4417.0,3.622368
...,...,...,...,...,...
226,Niue,2022-03-13,NaN,1937.0,NaN
227,Nauru,2022-07-11,NaN,12512.0,NaN
228,Jersey,2022-07-17,NaN,109618.0,NaN
229,Guernsey,2022-07-12,NaN,63065.0,NaN


In [11]:
#Maximum infection rate in US; finds out the time of that maximum infection

infected_rate_US_query ="""
    SELECT location, date, total_cases, new_cases, population, max((new_cases/population)*100) 
    as infection_rate_per_population 
    FROM cov_death WHERE location LIKE '%states'"""

ps.sqldf(infected_rate_US_query) 

,location,date,total_cases,new_cases,population,infection_rate_per_population
0,United States,2022-01-10,61801347.0,1383900.0,336997624.0,0.410656


In [12]:
#Most number of deaths across different countries as on 2022-07-24:

max_death_query = """
    SELECT date, location, max(total_deaths) as total_numb_of_deaths
    FROM cov_death WHERE continent is not null 
    GROUP BY 2
    ORDER BY 3 DESC 
    """

ps.sqldf(max_death_query) 

,date,location,total_numb_of_deaths
0,2022-07-24,United States,1026951.0
1,2022-07-24,Brazil,676964.0
2,2022-07-24,India,526074.0
3,2022-07-24,Russia,374411.0
4,2022-07-24,Mexico,327089.0
...,...,...,...
226,2022-07-24,Marshall Islands,NaN
227,2022-07-17,Jersey,NaN
228,2022-07-12,Guernsey,NaN
229,2022-06-18,Guam,NaN


In [13]:
#total_deaths_as_per_2022-07-24 continent wise

total_death_query ="""
    SELECT continent, MAX(cast(total_deaths as int)) 
    as total_death_by_covid from cov_death 
    WHERE continent is not null
    GROUP BY continent
    ORDER BY total_death_by_covid DESC
    """

ps.sqldf(total_death_query) 

,continent,total_death_by_covid
0,North America,1026951
1,South America,676964
2,Asia,526074
3,Europe,374411
4,Africa,101943
5,Oceania,11185
